In [5]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
import math 

sys.path.append(os.getcwd())
import tsa_utils
import deep_cnn
import flattened_cnn2
import mini_cnn

from shutil import copytree


In [9]:
DATA_PATH="/home/cody/workspace/tsa/images/"
CHECKPOINT_PATH="/home/cody/workspace/tsa_data/notebook1"
MODEL_ID="tsa_nb"

In [10]:
image_df = pd.read_csv(DATA_PATH + '/stage1_labels.csv')
image_df['zone'] = image_df['Id'].str.split("_", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()

ids = image_df["id"].unique()
ids.sort()
training_ids = ids

labels = image_df[image_df['id'].isin(training_ids)]
labels = labels.sort_values("id")

tensors_to_log =  {"probabilities": "softmax_tensor",
                    "actual":"labels"}


In [ ]:
with tf.Session() as sess:
	for zone in range(1,2):
		print("FITTING MODEL FOR ZONE " + str(zone))
		train_labels = labels[labels["zone"]=="Zone"+str(zone)]
		train_labels["class0"] = 0
		train_labels["class1"] = 0
		train_labels.loc[train_labels['Probability'] == 0, 'class0'] = 1
		train_labels.loc[train_labels['Probability'] == 1, 'class1'] = 1
		train_labels = np.reshape(np.array(train_labels[["class0","class1"]]), [-1,2])
		#model = deep_cnn.ZoneModel(MODEL_ID, training_ids, "Zone" + str(zone), None, None, DATA_PATH, train_labels, CHECKPOINT_PATH, localize=True)
		model = flattened_cnn2.ZoneModel(MODEL_ID, training_ids, "Zone" + str(zone), None, None, DATA_PATH, train_labels, CHECKPOINT_PATH, localize=False)
		#model.bootstrap_model()
		model.train_model(tensors_to_log, reuse=False)

FITTING MODEL FOR ZONE 1
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xa1b2050>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn 

/home/cody/anaconda3/envs/p2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/cody/anaconda3/envs/p2.7/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Tensor("conv1/BiasAdd:0", shape=(10, 16, 330, 256, 24), dtype=float32)
Tensor("conv2/Relu:0", shape=(10, 16, 330, 256, 24), dtype=float32)
Tensor("pool1/MaxPool3D:0", shape=(10, 1, 164, 127, 24), dtype=float32)
Tensor("pool2/MaxPool:0", shape=(10, 54, 42, 24), dtype=float32)
Tensor("conv3/BiasAdd:0", shape=(10, 27, 21, 24), dtype=float32)
Tensor("pool1_2/MaxPool:0", shape=(10, 13, 10, 24), dtype=float32)
Tensor("conv4/Relu:0", shape=(10, 13, 10, 24), dtype=float32)
Tensor("pool2_2/MaxPool:0", shape=(10, 6, 4, 24), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
